In [5]:
from IPython.display import display_javascript

def make_cell(s):

    display_javascript("""var t_cell = IPython.notebook.insert_cell_below()
    t_cell.set_text('<!--\\n' + t_cell.get_text() + '\\n--> \\n{}');
    var t_index = IPython.notebook.get_cells().indexOf(t_cell);
    IPython.notebook.to_markdown(t_index);
    IPython.notebook.get_cell(t_index).render();""".format(s.replace('\n','\\n')), raw=True)

# Introduction


So, you have somehow found micropython, fallen in love with it in an instance, broken your piggy bank, and run off, head over heels, to buy a pyboard. You have probably paid extra for the expedited shipping. Once the pyboard arrived, you became excited like a puppy with a bone. You played with the hardware, learnt how to access the accelerometer, switch, LEDs, and temperature sensor, and you have successfully communicated with other devices via the I2C, SPI, USART, or CAN interfaces. You have plugged the board in a computer, and driven someone crazy by emulating a seemingly disoriented mouse on it. You have written your own python functions, even compiled them into frozen modules, and burnt the whole damn thing onto the microcontroller. Then you have toyed with the on-board assembler, because you hoped that you could gain some astronomical factors in speed. (But you couldn't.)

And yet, after all this, you feel somewhat unsatisfied. You find that you want to access the periphery in a special way, or you need some fancy function that, when implemented in python itself, seems to consume too much RAM, and takes an eternity to execute, and assembly, especially with its limitations, is just far too awkward. Or perhaps, you are simply dead against making your code easily readable by writing everything in python, and you want to hide the magic, just for the heck of it. But you still want to retain the elegance of python. 

If, after thorough introspection and soul-searching, you have discovered these latter symptoms in yourself, you have two choices: either you despair, scrap your idea, and move on, or you learn how the heavy lifting behind the python facade is done, and spin your own functions, classes, and methods in C. As it turns out, it is not that hard, once you get the hang of it. The sole trick is to get the hang of it. The fact that the micropython code base is organised in a rather logical fashion certainly makes matters feasible. 

On the following pages I would like to show how new functionality can be added and exposed to the python interpreter. I will try to discuss all aspects of micropython in an approachable way. Each concept will be presented in a minimal implementation that you can compile right away, and try yourself. At the end of each chapter, I will list the discussed code in its entirety, so that copying and pasting does not involve copious amounts of work. The code, as well as the source of the documentation are also available under https://github.com/v923z/micropython-development.

I start out with a very simple module and slowly build upon it. At the end of the discussion, I will outline my version of a general-purpose math library, similar to numpy. In fact, when working on this math module, I realised that a decent programming guide to micropython is sourly missing, hence this documents. Obviously, numpy is a huge library, and we are not going to implement all aspects of it. But we will be able to define efficiently stored arrays on which we can do vectorised computations, work with matrices, invert and contract them, fit polynomials to measurement data, and get the Fourier transform of an arbitrary sequence. I hope you will enjoy the ride!

One last comment: I believe, all examples in this document could be implemented with little effort in python itself, and I am definitely not advocating the inclusion of such trivial cases in the firmware. I chose these examples on two grounds: First, they are all simple, almost primitive, but for this very reason, they demonstrate an idea without distraction. Second, having a piece of parallel python code is useful insofar as it tells us what to expect, and we can compare the results of the C implemetation to that of the native firmware.

# Setting up the environment, compiling the code


Beginning with the 1.10 version of micropython, it became quite simple to add a user-defined C module to the firmware. 


-------------------------------

```bash

make USER_C_MODULES=../../../modules CFLAGS_EXTRA=-DMODULE_EXAMPLE_ENABLED=1 all
```
or 

```bash

make USER_C_MODULES=../../../modules all
```


The micropython codebase itself is set up a rather modular way. If you look at the top level directories,

In [5]:
!ls ../../micropython/

ACKNOWLEDGEMENTS    docs      extmod   logo	  py	     tools
CODECONVENTIONS.md  drivers   lib      mpy-cross  README.md
CONTRIBUTING.md     examples  LICENSE  ports	  tests


at least two are of particular interest. Namely, `/py/`, where the python interpreter is implemented, and `/ports/`, which contains the hardware-specific files. 

# micropython internals

Before exploring the exciting problem of micropython function implementation in C, we should first understand how python objects are stored and treated at the firmware level. 

## Object representation

Whenever you write 

```python
>>> a = 1
>>> b = 1
>>> a + b
```

on the python console, first the two new variables, `a`, and `b` are created and a reference to them is stored in memory. Then the value of 1 will be associated with these variables. In the last line, when the sum is to be computed, the interpreter somehow has to figure out, how to decipher the values stored in `a`, and `b`: in the RAM, these two variables are just bytes, but depending on the type of the variable, different meanings will be associated with these bytes. Since the type cannot be known at compile time, there must be a mechanism for keeping stock of this extra piece of information. This is, where `mp_obj_t`, defined in `obj.h`, takes centre stage. 

If you look at any of the C functions that are exposed to the python interpreter, you will see something like this

```c
mp_obj_t some_function(mp_obj_t some_variable, ...) {
    // some_variable is converted to fundamental C types (bytes, ints, floats, pointers, structures, etc.)
    ...
}
```
Variables of type `mp_obj_t` are passed to the function, and the function returns the results as an object of type `mp_obj_t`. So, what is all this fuss this about? Basically, `mp_obj_t` is nothing but an 8-byte segment of the memory, where all concrete objects are encoded. There can be various object encodings. E.g., in the `A` encoding, integers are those objects, whose rightmost bit in this 8-byte representation is set to 1, and the value of the integer can then be retrieved by shifting these 8 bytes by one to the right. In the `B` encoding, the variable is an integer, if its value is 1, when ANDed with 3, and the value will be returned, if we shift the 8 bytes by two to the right. 

Fortunately, we do not have to be concerned with the representations and the shifts, because there are pre-defined macros for such operations. So, if we want to find out, whether `some_variable` is an integer, we can inspect the value of the Boolean 

```c
MP_OBJ_IS_SMALL_INT(some_variable)
```

The integer value stored in `some_variable` can then be gotten by calling `MP_OBJ_SMALL_INT_VALUE`: 

```c
int value_of_some_variable = MP_OBJ_SMALL_INT_VALUE(some_variable);
```

These decoding steps take place somewhere in the body of `some_function`, before we start working with native C types. Once we are done with the calculations, we have to return an `mp_obj_t`, so that the interpreter can handle the results (e.g., show it on the console, or pipe it to the next instruction). In this case, the encoding is done by calling 

```c
mp_obj_new_int(value_of_some_variable)

```

More generic types can be treated with the macro `MP_OBJ_IS_TYPE`, which takes the object as the first, and a pointer to the type as the second argument. So, if you want to find out, whether `some_variable` is a tuple, you could apply the `MP_OBJ_IS_TYPE` macro, 

```c
MP_OBJ_IS_TYPE(myobject, &mp_type_tuple)
```

While the available types can be found in `obj.h`, they all follow the `mp_type_` + python type pattern, so in most cases, it is not even necessary to look them up. We should also note that, when defined properly, `MP_OBJ_IS_TYPE` can be called on objects, that are defined in the user module itself, i.e., you can have your own data type, and 

```c
MP_OBJ_IS_TYPE(myobject, &my_type)
```

will just work. We return to this question later. 

## Constants

At this point, we should mention python constants,`True`, `False`, `None` and the like are also defined in `obj.h`. These are objects of type `mp_obj_t`, as almost anything else, so you can return them from a function, when the function is meant to return directly to the interpreter.

# Developing your own module

Having seen, what the python objects look like to the interpreter, we can start with out explorations in earnest. We begin by adding a simple module to the python interpreter. The module will have a single function that takes two numbers, and adds them. I know that this is the most exciting thing since sliced bread, and you always wondered, why there isn't a built-in python function for such an exciting task. Well, wonder no more! From now on, *your* micropython will have one. 

First I show the file in its entirety (20 something lines all in all), and then discuss the parts.

```c
#include "py/obj.h"
#include "py/runtime.h"

STATIC mp_obj_t simplefunction_add_ints(mp_obj_t a_obj, mp_obj_t b_obj) {
    int a = mp_obj_get_int(a_obj);
    int b = mp_obj_get_int(b_obj);
    return mp_obj_new_int(a + b);
}

STATIC MP_DEFINE_CONST_FUN_OBJ_2(simplefunction_add_ints_obj, simplefunction_add_ints);

STATIC const mp_rom_map_elem_t simplefunction_module_globals_table[] = {
    { MP_ROM_QSTR(MP_QSTR___name__), MP_ROM_QSTR(MP_QSTR_simplefunction) },
    { MP_ROM_QSTR(MP_QSTR_add_ints), MP_ROM_PTR(&simplefunction_add_ints_obj) },
};
STATIC MP_DEFINE_CONST_DICT(simplefunction_module_globals, simplefunction_module_globals_table);

const mp_obj_module_t simplefunction_user_cmodule = {
    .base = { &mp_type_module },
    .globals = (mp_obj_dict_t*)&simplefunction_module_globals,
};

MP_REGISTER_MODULE(MP_QSTR_simplefunction, simplefunction_user_cmodule, MODULE_SIMPLEFUNCTION_ENABLED);

```

You should be able to compile the module above by calling 

```bash
make USER_C_MODULES=../../../usermod/ all
```

## Header files

A module will not be too useful without at least two includes: `py/obj.h`, where all the relevant constants and macros are defined, and `py/runtime.h`, which contains the declaration of the interpreter. 

## Defining user functions

After including the necessary headers, we define the function that is going to do the heavy lifting. By passing variables of `mp_obj_t` type, we make sure that the function will be able to accept values from the python console. If you happen to have an internal helper function in your module that is not exposed in python, you can pass whatever type you need. Similarly, by returning an object of `mp_obj_t` type, we make the results visible to the interpreter, i.e., we can assign the value returned to variables. 

The downside of passing `mp_obj_t`s around is that you cannot simply assign them to usual C variables, i.e., when you want to operate on them, you have to extract the values first. This is why we have to invoke the `mp_obj_get_int()` function, and conversely, before returning the results, we have to do a type conversion to `mp_obj_t` by calling `mp_obj_new_int()`. These are the decoding/encoding steps that we discussed above.

### Referring to user functions

We have now a function that should be sort of OK, but the python interpreter still cannot work with. For that, we have to turn our function into a function object. This is what happens in the line

```c
STATIC MP_DEFINE_CONST_FUN_OBJ_2(simplefunction_add_ints_obj, simplefunction_add_ints);
```

The first argument is the name of the function object to which our actual function, the last argument, will be bound. Now, these `MP_DEFINE_CONST_FUN_OBJ_*` macros, defined in the header file `py/obj.h` (one more reason not to forget about `py/obj.h`), come in seven flavours, depending on what kind of, and how many arguments the function is supposed to take. In the example above, our function is meant to take two arguments, hence the 2 at the end of the macro name. Functions with 0 to 4 arguments can be bound in this way.

But what, if you want a function with more than four arguments, as is the case many a time in python? Under such circumstances, one can make use of the 

```c
MP_DEFINE_CONST_FUN_OBJ_VAR(obj_name, n_args_min, fun_name)
```

macro, where the second argument, an integer, gives the minimum number of arguments. The number of arguments can be bound from above by wrapping the function with 

```c
MP_DEFINE_CONST_FUN_OBJ_VAR_BETWEEN(obj_name, n_args_min, n_args_max, fun_name) 
```

Later we will see, how we can define functions that can also take keyword arguments.

At this point, we are more or less done with the C implementation of our function, but we still have to expose it. This we do by adding a table, an array of key/value pairs to the globals of our module, and bind the table to the `_module_globals` variable by applying the `MP_DEFINE_CONST_DICT` macro. This table should have at least one entry, the name of the module, which is going to be stored in the string `MP_QSTR___name__`. 

The other keys of the table are the pointers to the functions that we implemented, and the names that we want to call these functions in python itself. So, in the example below, our `simplefunction_add_ints` function will be invoked, when we call `add_ints` in the console.


```c
STATIC const mp_rom_map_elem_t simplefunction_module_globals_table[] = {
    { MP_ROM_QSTR(MP_QSTR___name__), MP_ROM_QSTR(MP_QSTR_simplefunction) },
    { MP_ROM_QSTR(MP_QSTR_add_ints), MP_ROM_PTR(&simplefunction_add_ints_obj) },
};
STATIC MP_DEFINE_CONST_DICT(simplefunction_module_globals, simplefunction_module_globals_table);
```
This three-step pattern is common to all function implementations, so I repeat it here: 

1. implement the function
2. then turn it into a function object
3. and finally, register the function in the name space of the module

It doesn't matter, whether our function takes positional arguments, or keyword argument, or both, these are the required steps.

Having defined the function object, we have finally to register the function with 

```c
MP_REGISTER_MODULE(MP_QSTR_simplefunction, simplefunction_user_cmodule, MODULE_SIMPLEFUNCTION_ENABLED);
```

This last line is particularly useful, because by setting the `MODULE_SIMPLEFUNCTION_ENABLED` variable in `mpconfigport.h`, you can selectively exclude modules from the compilation, i.e., if in `mpconfigport.h`, which should be in the root directory of the port you want to compile for, 

```c
#define MODULE_SIMPLEFUNCTION_ENABLED (1)
```
then `simplefunction` will be included in the firmware, while with

```c
#define MODULE_SIMPLEFUNCTION_ENABLED (0)
```

the module will be dropped, even though the source is in your modules folder.

## Function names

Whenever you want your function to be visible from the interpreter, you have to add the function object to a dictionary. The dictionary 

```c
STATIC const mp_rom_map_elem_t test_locals_dict_table[] = {
	{ MP_ROM_QSTR(MP_QSTR_somefunction), MP_ROM_PTR(&test_somefunction_obj) },
};


STATIC MP_DEFINE_CONST_DICT(test_locals_dict, test_locals_dict_table);
```

## Parsing arguments

## Parsing keyword arguments

One of the most useful features of python is that functions can accept positional as well as keyword arguments, thereby providing a very flexible interface for feeding values to a function. In this section, we will see how we should set up our function, so that it can interpret what we passed in. 

It does not matter, whether we have positional or keyword arguments, at one point, the interpreter has to turn all arguments into a deterministic sequence of objects. We stipulate this sequence in the constant variable called `allowed_args[]`. This is an array of type `mp_arg_t`, which is nothing but a structure with two `uint16` values, and a union named `mp_arg_val_t`. This union holds the default value and the type of the variable that we want to pass. The `mp_arg_t` structure, defined in `runtime.h`, looks like this:

```c
typedef struct _mp_arg_t {
    uint16_t qst;
    uint16_t flags;
    mp_arg_val_t defval;
} mp_arg_t;
```

The last member, `mp_arg_val_t` is 

```c
typedef union _mp_arg_val_t {
    bool u_bool;
    mp_int_t u_int;
    mp_obj_t u_obj;
    mp_rom_obj_t u_rom_obj;
} mp_arg_val_t;
```

### Keyword arguments with numerical values

In our first example `keywordfunction.c`, we define a summing function with two arguments. 

```c
#include "py/obj.h"
#include "py/runtime.h"
#include "py/builtin.h"

STATIC mp_obj_t keywordfunction_add_ints(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args) {

	static const mp_arg_t allowed_args[] = {
		{ MP_QSTR_a, MP_ARG_INT | MP_ARG_REQUIRED, {.u_int = 0} }, 
        { MP_QSTR_b, MP_ARG_KW_ONLY | MP_ARG_INT | MP_ARG_REQUIRED, {.u_int = 0} },
    };

    mp_arg_val_t args[MP_ARRAY_SIZE(allowed_args)];
    mp_arg_parse_all(n_args, pos_args, kw_args, MP_ARRAY_SIZE(allowed_args), allowed_args, args);
	int16_t a = args[0].u_int;
	int16_t b = args[1].u_int;
	printf("a = %d, b = %d\n", a, b);
	return mp_obj_new_int(a + b);
}

STATIC MP_DEFINE_CONST_FUN_OBJ_KW(keywordfunction_add_ints_obj, 2, keywordfunction_add_ints);

STATIC const mp_rom_map_elem_t keywordfunction_module_globals_table[] = {
    { MP_ROM_QSTR(MP_QSTR___name__), MP_ROM_QSTR(MP_QSTR_keywordfunction) },
    { MP_ROM_QSTR(MP_QSTR_add_ints), (mp_obj_t)&keywordfunction_add_ints_obj },
};

STATIC MP_DEFINE_CONST_DICT(keywordfunction_module_globals, keywordfunction_module_globals_table);

const mp_obj_module_t keywordfunction_user_cmodule = {
    .base = { &mp_type_module },
    .globals = (mp_obj_dict_t*)&keywordfunction_module_globals,
};

MP_REGISTER_MODULE(MP_QSTR_keywordfunction, keywordfunction_user_cmodule, MODULE_KEYWORDFUNCTION_ENABLED);
```

One convenience of a function with keyword arguments is that we do not have to care about the arguments in the C implementation: the argument list is always the same, and it is passed in by the interpreter: the number of arguments of the python function, an array with the positional arguments, and a map for the keyword arguments. 

After parsing the arguments with `mp_arg_parse_all`, whatever was at the zeroth position of `allowed_args[]` will be called `args[0]`, the object at the first position of `allowed_args[]` will be turned into `args[1]`, and so on. 

This is, where we also define, what the name of the keyword argument is going to be: whatever comes after `MP_QSTR_`. But hey, presto! The name should be an integer with 16 bits, shouldn't it? After all, this is the first member of `mp_arg_t`. So what is going on here? Well, for the efficient use of RAM, all MP_QSTRs are turned into `unint16_t` internally. This applies not only to the names in functions with keyword arguments, but also for module and function names, in the `_module_globals_table[]`.

The second member of the `mp_arg_t` structure is the flags that determine, e.g., whether the argument is required, if it is of integer or `mp_obj_t` type, and whether it is positional or a keyword argument. These flags can be combined by ORing them, as we done in the example above. 

The last member in `mp_arg_t` is the default value. Since this is a member variable, when we make use of it, we have to extract the value by adding `.u_int` to the argument. 

When turning our function into a function object, we have to call a special macro, `MP_DEFINE_CONST_FUN_OBJ_KW`, defined in `obj.h`, which is somewhat similar to `MP_DEFINE_CONST_FUN_OBJ_VAR`: in addition to the function object and the function, one also has to specify the minimum number of arguments in the python function.

Other examples on passing keyword arguments can be found in some of the hardware implementation files, e.g., `ports/stm32/pyb_i2c.c`, or `ports/stm32/pyb_spi.c`.

### Keyword arguments with arbitrary values

We have seen how integer values can be extracted from keyword arguments, but unfortunately, that method is going to get you only that far. What if we want to pass something more complicated, in particular a string, or a list, or some other non-trivial python type? This is what we are going to look at now. 

# Working with classes

Of course, python would not be python without classes. A module can also include the implementation of classes. The procedure is similar to what we have already seen in the context of standard functions, except that we have to define a structure that holds at least a string with the name of the class, a pointer to the initialisation and printout functions, and a local dictionary. A typical structure would look like

```c
STATIC const mp_rom_map_elem_t test_locals_dict_table[] = {
	{ MP_ROM_QSTR(MP_QSTR_method1), MP_ROM_PTR(&test_method1_obj) },
	{ MP_ROM_QSTR(MP_QSTR_method2), MP_ROM_PTR(&test_method2_obj) },
    ...                                                           
}

const mp_obj_type_t simpleclass_type = {
	{ &mp_type_type },
	.name = MP_QSTR_simpleclass,
	.print = simpleclass_print,
	.make_new = simpleclass_make_new,
	.locals_dict = (mp_obj_dict_t*)&simpleclass_locals_dict,
};
```

The locals dictionary, `.locals_dict`, contains all user-facing methods and constants of the class, while the `myclass_type` structure's `name` member is what our class is going to be called. `.print` is roughly the equivalent of `__str__`, `.make_new` is the C name for `__init__`. 

In order to see how this all works, we are going to implement a very simple class, which holds two integer variables, and has a method that returns the sum of these two variables. In python, a possible realisation could look like this:

In [10]:
class myclass:
    
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    def mysum(self):
        return self.a + self.b
    
    
A = myclass(1, 2)
A.mysum()

3

In addition to the class implementation above and to show how class methods and regular functions can live in the same module, we will also have a function, which is not bound to the class itself, and which adds the two components in the class, i.e., that is similar to 

In [11]:
def add(class_instance):
    return class_instance.a + class_instance.b

add(A)

3

(Note that retrieving values from the class in this way is not exactly elegant. We would usually implement a getter method for that.)

```c
#include "py/runtime.h"
#include "py/obj.h"

typedef struct _simpleclass_myclass_obj_t {
	mp_obj_base_t base;
	int16_t a;
	int16_t b;
} simpleclass_myclass_obj_t;

const mp_obj_type_t simpleclass_myclass_type;

STATIC void myclass_print(const mp_print_t *print, mp_obj_t self_in, mp_print_kind_t kind) {
	(void)kind;
	simpleclass_myclass_obj_t *self = MP_OBJ_TO_PTR(self_in);
	mp_print_str(print, "myclass(");
	printf("%d, ", self->a);
	printf("%d)", self->b);
}

STATIC mp_obj_t myclass_make_new(const mp_obj_type_t *type, size_t n_args, size_t n_kw, const mp_obj_t *args) {
	mp_arg_check_num(n_args, n_kw, 2, 2, true);
	simpleclass_myclass_obj_t *self = m_new_obj(simpleclass_myclass_obj_t);
	self->base.type = &simpleclass_myclass_type;
	self->a = mp_obj_get_int(args[0]);
	self->b = mp_obj_get_int(args[1]);
	return MP_OBJ_FROM_PTR(self);
}

// Class methods
STATIC mp_obj_t myclass_sum(mp_obj_t self_in) {
	simpleclass_myclass_obj_t *self = MP_OBJ_TO_PTR(self_in);
	return mp_obj_new_int(self->a + self->b);
}

MP_DEFINE_CONST_FUN_OBJ_1(myclass_sum_obj, myclass_sum);

STATIC const mp_rom_map_elem_t myclass_locals_dict_table[] = {
	{ MP_ROM_QSTR(MP_QSTR_mysum), MP_ROM_PTR(&myclass_sum_obj) },
};

STATIC MP_DEFINE_CONST_DICT(myclass_locals_dict, myclass_locals_dict_table);

const mp_obj_type_t simpleclass_myclass_type = {
	{ &mp_type_type },
	.name = MP_QSTR_simpleclass,
	.print = myclass_print,
	.make_new = myclass_make_new,
	.locals_dict = (mp_obj_dict_t*)&myclass_locals_dict,
};

// Module functions
STATIC mp_obj_t simpleclass_add(const mp_obj_t o_in) {
	simpleclass_myclass_obj_t *class_instance = MP_OBJ_TO_PTR(o_in);
	return mp_obj_new_int(class_instance->a + class_instance->b);
}

MP_DEFINE_CONST_FUN_OBJ_1(simpleclass_add_obj, simpleclass_add);

STATIC const mp_map_elem_t simpleclass_globals_table[] = {
	{ MP_OBJ_NEW_QSTR(MP_QSTR___name__), MP_OBJ_NEW_QSTR(MP_QSTR_simpleclass) },
	{ MP_OBJ_NEW_QSTR(MP_QSTR_myclass), (mp_obj_t)&simpleclass_myclass_type },	
	{ MP_OBJ_NEW_QSTR(MP_QSTR_add), (mp_obj_t)&simpleclass_add_obj },
};

STATIC MP_DEFINE_CONST_DICT (
	mp_module_simpleclass_globals,
	simpleclass_globals_table
);

const mp_obj_module_t simpleclass_user_cmodule = {
	.base = { &mp_type_module },
	.globals = (mp_obj_dict_t*)&mp_module_simpleclass_globals,
};

MP_REGISTER_MODULE(MP_QSTR_simpleclass, simpleclass_user_cmodule, MODULE_SIMPLECLASS_ENABLED);
```

In `my_print`, we used the C function `printf`, but better options are also available. `mpprint.c` has a number of methods for printing all kinds of python objects. 

## Class special methods

Python has a number of special methods, which will make a class behave as a native object. So, e.g., if a class implements the `__add__(self, other)` method, then instances of that class can be added with the `+` operator. Here is an example in python:

In [4]:
class Adder:
    
    def __init__(self, value):
        self.value = value
        
    def __add__(self, other):
        self.value = self.value + other.value
        return self

a = Adder(1)
b = Adder(2)

c = a + b
c.value

3

Note that, while the above example is not particularly useful, it proves the point: upon calling the `+` operator, the values of `a`, and `b` are added. If we had left out the implementation of the `__add__` method, the python interpreter would not have a clue as to what to do with the objects. You can see for yourself, how sloppiness makes itself manifest:

In [5]:
class Adder:
    
    def __init__(self, value):
        self.value = value

a = Adder(1)
b = Adder(2)

c = a + b
c.value

TypeError: unsupported operand type(s) for +: 'Adder' and 'Adder'

Indeed, we do not support the `+` operator.

Now, the problem is that in the C implementation, these special methods have to be treated in a special way. The naive approach would be to add the pointer to the function to the locals dictionary as 

```c
STATIC const mp_rom_map_elem_t test_locals_dict_table[] = {
	{ MP_ROM_QSTR(MP_QSTR___add__), MP_ROM_PTR(&test_add_obj) },
};
```
but that would not work. Well, this is not entirely true: the `+` operator would not work, but one could still call the method explicitly as

```python
a = Adder(1)
b = Adder(2)

a.__add__(b)

```

Before we actually add the `+` operator to our class, we should note that there are two kinds of special methods, namely the unary and the binary operators. 

In the first group are those, whose sole argument is the class instance itself. Two frequently used cases are the length operator, `len`, and `bool`. So, e.g., if your class implements the `__len__(self)` method, and the method returns an integer, then you can call the `len` function in the console

```python
len(myclass)
```

In the second category of operators are those, which require a left, as well as a right hand side: the left hand side is the class instance itself, while the right hand side can, in principle, be another instance of the same class, or some other type. An example for this was the `__add__` method in our `Adder` class. To prove that the right hand side needn't be of the same type, think of the *multiplication* of lists: 

```python
[1, 2, 3]*5
```

is perfectly valid, and has a well-defined meaning. The complete list of unary, as well as binary operators can be found in `runtime.h`. 

The module below implements five special methods altogether. Two unary, namely, `bool`, and `len`, and three binary operators, `==`, `+`, and `*`. Since the addition and multiplication will return a new instance of `myclass`, we define a new function, `create_new_class`, that, well, creates a new instance of `myclass`, and initialises the members with the two input arguments. This function will also be called in the class initialisation function, `myclass_make_new`, immediately after the argument checking. 

When implementing the operators, we have to keep a couple of things in mind. First, the `specialclass_myclass_type` has to be extended with the two methods, `.unary_op`, and `.binary_op`, where `.unary_op` is equal to the function that handles the unary operation (`specialclass_unary_op` in the example below), and `.binary_op` is equal to the function that deals with binary operations (`specialclass_binary_op` below). These two functions have the signatures 

```c
STATIC mp_obj_t specialclass_unary_op(mp_unary_op_t op, mp_obj_t self_in)
```
and

```c
STATIC mp_obj_t specialclass_binary_op(mp_binary_op_t op, mp_obj_t lhs, mp_obj_t rhs)
```
respectively, and we have to inspect the value of `op` in the implementation. This is done in the two `switch` statements. 

One more comment: if `.unary_op`, or `.binary_op` are defined for the class, then the handler function must have an implementation of all operators. This doesn't necessarily mean that you have to have all cases in the `switch`, but if you haven't, then there must be a default case with a reasonable return value, e.g., `MP_OBJ_NULL`, or `mp_const_none`, so as to indicate that that particular method is not available. 

```c
#include "py/runtime.h"
#include "py/obj.h"
#include "py/binary.h"

typedef struct _specialclass_myclass_obj_t {
	mp_obj_base_t base;
	int16_t a;
	int16_t b;
} specialclass_myclass_obj_t;

const mp_obj_type_t specialclass_myclass_type;

STATIC void myclass_print(const mp_print_t *print, mp_obj_t self_in, mp_print_kind_t kind) {
	(void)kind;
	specialclass_myclass_obj_t *self = MP_OBJ_TO_PTR(self_in);
	mp_print_str(print, "myclass(");
	printf("%d, ", self->a);
	printf("%d)", self->b);
}

mp_obj_t create_new_myclass(uint16_t a, uint16_t b) {
	specialclass_myclass_obj_t *out = m_new_obj(specialclass_myclass_obj_t);
	out->base.type = &specialclass_myclass_type;
	out->a = a;
	out->b = b;
	return MP_OBJ_FROM_PTR(out);
}

STATIC mp_obj_t myclass_make_new(const mp_obj_type_t *type, size_t n_args, size_t n_kw, const mp_obj_t *args) {
	mp_arg_check_num(n_args, n_kw, 2, 2, true);
	return create_new_myclass(mp_obj_get_int(args[0]), mp_obj_get_int(args[1]));
}

STATIC const mp_rom_map_elem_t myclass_locals_dict_table[] = {
};

STATIC MP_DEFINE_CONST_DICT(myclass_locals_dict, myclass_locals_dict_table);

STATIC mp_obj_t specialclass_unary_op(mp_unary_op_t op, mp_obj_t self_in) {
	specialclass_myclass_obj_t *self = MP_OBJ_TO_PTR(self_in);
    switch (op) {
        case MP_UNARY_OP_BOOL: return mp_obj_new_bool((self->a > 0) && (self->b > 0));
        case MP_UNARY_OP_LEN: return mp_obj_new_int(2);
        default: return MP_OBJ_NULL; // op not supported
    }
}

STATIC mp_obj_t specialclass_binary_op(mp_binary_op_t op, mp_obj_t lhs, mp_obj_t rhs) {
    specialclass_myclass_obj_t *left_hand_side = MP_OBJ_TO_PTR(lhs);
	specialclass_myclass_obj_t *right_hand_side = MP_OBJ_TO_PTR(rhs);
	switch (op) {
		case MP_BINARY_OP_EQUAL:
			return mp_obj_new_bool((left_hand_side->a == right_hand_side->a) && (left_hand_side->b == right_hand_side->b));
		case MP_BINARY_OP_ADD:
			return create_new_myclass(left_hand_side->a + right_hand_side->a, left_hand_side->b + right_hand_side->b);
		case MP_BINARY_OP_MULTIPLY:
			return create_new_myclass(left_hand_side->a * right_hand_side->a, left_hand_side->b * right_hand_side->b);
		default:
			return MP_OBJ_NULL; // op not supported
	}
}

const mp_obj_type_t specialclass_myclass_type = {
	{ &mp_type_type },
	.name = MP_QSTR_specialclass,
	.print = myclass_print,
	.make_new = myclass_make_new,
	.unary_op = specialclass_unary_op, 
	.binary_op = specialclass_binary_op,
	.locals_dict = (mp_obj_dict_t*)&myclass_locals_dict,
};

STATIC const mp_map_elem_t specialclass_globals_table[] = {
	{ MP_OBJ_NEW_QSTR(MP_QSTR___name__), MP_OBJ_NEW_QSTR(MP_QSTR_specialclass) },
	{ MP_OBJ_NEW_QSTR(MP_QSTR_myclass), (mp_obj_t)&specialclass_myclass_type },	
};

STATIC MP_DEFINE_CONST_DICT (
	mp_module_specialclass_globals,
	specialclass_globals_table
);

const mp_obj_module_t specialclass_user_cmodule = {	
	.base = { &mp_type_module },
	.globals = (mp_obj_dict_t*)&mp_module_specialclass_globals,
};

MP_REGISTER_MODULE(MP_QSTR_specialclass, specialclass_user_cmodule, MODULE_SPECIALCLASS_ENABLED);

```

## Defining constants

Constants can be added to the locals dictionary as any other object. So, e.g., if we wanted to define the constant MAGIC, we could do that as follows

```c

#define MAGIC 42

STATIC const mp_rom_map_elem_t test_locals_dict_table[] = {
	{ MP_ROM_QSTR(MP_QSTR_MAGIC), MP_ROM_INT(MAGIC) },
};
```
The constant would then be accessible in the interpreter as

```python

import test

test.MAGIC
```

## The complete code

Having seen how components of a class should be implemented, we can bring all parts together. We will define a class that has two methods, one with positional arguments, and one with keyword arguments, has a class-specific constant, and can deal with the `+` and `*` operators. You should be able to drop this code into your user-defined modules directory, and compile it without difficulties.

```c

```

# Dealing with iterables


In python, an iterable is basically an object that you can have in a `for` loop:

```python
for item in my_iterable:
    print(item)
```

Amongs others, lists, tuples, and ranges are iterables, as are strings. The key is that these object have a special internal method, and iterator, attached to them. This iterator is responsible for keeping track of the index during the iteration, and serving the objects in the iterable one by one to the `for` loop. When writing our own iterable, we will look under the hood, and see how this all works at the C level. For now, we are going to discuss only, how we can *consume* the content of an iterable in the C code. 

## Iterating over built-in types

In order to demonstrate the use of an iterator, we are going to write a function that sums the square of the values in an iterable. In python, the function could look like 

```python
def sumsq(iterable):
    return sum([item**2 for item in iterable])
```
In C, the key is in the snippet

```c
mp_obj_iter_buf_t iter_buf;
mp_obj_t item, iterable = mp_getiter(o_in, &iter_buf);
while ((item = mp_iternext(iterable)) != MP_OBJ_STOP_ITERATION) {
    // do something with the item just retrieved
}
```
`o_in` is the argument of our function. We have to turn this object into an iterable by passing it into the `mp_getiter` function. This function also needs a buffer object that is of type `mp_obj_iter_buf_t`. The `mp_getiter` 


The buffer type is defined in `obj.h` as 

```c
typedef struct _mp_obj_iter_buf_t {
    mp_obj_base_t base;
    mp_obj_t buf[3];
} mp_obj_iter_buf_t;

```



If you want to find out, whether a particular variable is an iterable, you can 


`objlist.c`, `objtuple.c`, `objarray.c`, `objrange.c` etc.

# Creating new types

Sometimes it might be necessary to define your own data type. It is really nothing but a C structure with a couple of special fields. Take the following type definition, which could be regarded as the three Cartesian components of a vector in three-dimensional space: 

```c
typedef struct _vector_obj_t {
	mp_obj_base_t base;
	float x, y, z;
} vector_obj_t;
```

Now, in order to see, how we can work with this structure, we are going to write a class that simply stores the three values, and has a method called `length`, returning the length of the vector. 

# Profiling

There are times, when you might want to find out how long a particular operation takes. If you are interested in the execution time of a complete function, you can measure it simply by making use of the python interpreter

```python

from utime import ticks_us, diff_ticks

now = ticks_us
run_my_function()
then = diff_ticks(ticks_us(), now)

print("function run_my_function() took %d us to run"%then)

```

In fact, since our function is flanked by two other statements, this construct easily lends itself to a decorator implementation. 

(If you need an even better estimate, you can yank run_my_function(): in this way, you would get the costs of measuring time itself:

```python

from utime import ticks_us, diff_ticks

now = ticks_us
then = diff_ticks(ticks_us(), now)

print("the time measurement took %d us"%then)

```
Then you subtract the results of the second measurement from those of the first.)